In [22]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=2ceed740892bb4cd964e519a0e5825d617be157ba86059812113efdd60bb741f
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [13]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:///intern_tracking.db')

# Load CSV data into SQL
pd.read_csv("/content/interns.csv").to_sql('interns', engine, if_exists='replace')
pd.read_csv("/content/tasks.csv").to_sql('tasks', engine, if_exists='replace')

1000

In [16]:
import pandas as pd
from datetime import datetime

def calculate_kpis():
    # Load data
    interns = pd.read_csv("/content/interns.csv")
    tasks = pd.read_csv("/content/tasks.csv")

    # Current month filter
    current_month = datetime.now().strftime("%Y-%m")

    # Task completion rate
    task_metrics = tasks.groupby('intern_id').agg(
        total_tasks=('task_id', 'count'),
        completed_tasks=('status', lambda x: (x == 'Completed').sum()),
        avg_completion_days=('completion_date', lambda x:
            (pd.to_datetime(x) - pd.to_datetime(tasks['assigned_date'])).mean().days)
    ).reset_index()

    # Merge with intern data
    performance = interns.merge(task_metrics, on='intern_id')
    performance['completion_rate'] = (performance['completed_tasks'] / performance['total_tasks']) * 100

    # Save results
    performance.to_csv("/content/current_performance.csv", index=False)
    return performance

if __name__ == "__main__":
    calculate_kpis()

In [27]:
from fpdf import FPDF
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def create_pdf_report():
    data = pd.read_csv("/content/current_performance.csv")

    # Create visualization
    plt.figure(figsize=(10, 6))
    sns.barplot(data=data, x='name', y='completion_rate')
    plt.xticks(rotation=45)
    plt.title("Intern Completion Rates")
    plt.tight_layout()
    plt.savefig("/content/completion_rates.png")
    plt.close()

    # Generate PDF
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, "Monthly Intern Performance Report", ln=True)

    pdf.set_font("Arial", '', 12)
    pdf.cell(0, 10, f"Generated on: {pd.Timestamp.now().date()}", ln=True)

    # Add table
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(40, 10, "Name", border=1)
    pdf.cell(40, 10, "Department", border=1)
    pdf.cell(40, 10, "Completion %", border=1)
    pdf.cell(40, 10, "Avg Days", border=1)
    pdf.ln()

    pdf.set_font("Arial", '', 10)
    for _, row in data.iterrows():
        pdf.cell(40, 10, row['name'], border=1)
        pdf.cell(40, 10, row['department'], border=1)
        pdf.cell(40, 10, f"{row['completion_rate']:.1f}%", border=1)
        pdf.cell(40, 10, f"{row['avg_completion_days']:.1f}", border=1)
        pdf.ln()

    # Add image
    pdf.image("/content/completion_rates.png", x=10, y=pdf.get_y() + 10, w=180)

    pdf.output("/content/monthly_report.pdf")

if __name__ == "__main__":
    create_pdf_report()

In [28]:
REPORT_RECIPIENTS = ["manager@company.com", "hr@company.com"]
SMTP_CONFIG = {
    "server": "smtp.gmail.com",
    "port": 587,
    "username": "reports@company.com",
    "password": "yourpassword"
}

In [29]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

# Directly define REPORT_RECIPIENTS and SMTP_CONFIG here instead of importing
REPORT_RECIPIENTS = ["manager@company.com", "hr@company.com"]
SMTP_CONFIG = {
    "server": "smtp.gmail.com",
    "port": 587,
    "username": "reports@company.com",
    "password": "yourpassword"  # Replace with your actual password
}

def send_email():
    msg = MIMEMultipart()
    msg['From'] = SMTP_CONFIG['username']
    msg['To'] = ", ".join(REPORT_RECIPIENTS)
    msg['Subject'] = "Monthly Intern Performance Report"

    body = """
    <h1>Intern Performance Report</h1>
    <p>Attached is this month's performance report.</p>
    """
    msg.attach(MIMEText(body, 'html'))

    with open("reports/monthly_report.pdf", "rb") as f:
        attach = MIMEApplication(f.read(), _subtype="pdf")
        attach.add_header('Content-Disposition', 'attachment', filename="report.pdf")
        msg.attach(attach)

    with smtplib.SMTP(SMTP_CONFIG['server'], SMTP_CONFIG['port']) as server:
        server.starttls()
        server.login(SMTP_CONFIG['username'], SMTP_CONFIG['password'])
        server.send_message(msg)

In [31]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Load data
performance = pd.read_csv("/content/current_performance.csv")

# Set style
sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))

# Create plot
ax = sns.histplot(data=performance, x="completion_rate", bins=10, kde=True)
plt.title("Distribution of Task Completion Rates", fontsize=14)
plt.xlabel("Completion Rate (%)", fontsize=12)
plt.ylabel("Number of Interns", fontsize=12)

# Add average line
avg_rate = performance['completion_rate'].mean()
plt.axvline(avg_rate, color='red', linestyle='--',
            label=f'Average: {avg_rate:.1f}%')
plt.legend()

plt.tight_layout()
plt.savefig("/content/completion_distribution.png", dpi=300)
plt.close()

In [33]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [41]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import base64
from io import BytesIO

# Load your data
performance = pd.read_csv("/content/current_performance.csv")
tasks = pd.read_csv("/content/tasks.csv")
quality = pd.read_csv("/content/project_quality.csv")
feedback = pd.read_csv("/content/mentor_feedback.csv")

# Preprocess data
tasks['assigned_date'] = pd.to_datetime(tasks['assigned_date'])
tasks['completion_date'] = pd.to_datetime(tasks['completion_date'])
tasks['days_to_complete'] = (tasks['completion_date'] - tasks['assigned_date']).dt.days
tasks['month'] = tasks['assigned_date'].dt.to_period('M').astype(str)
feedback['month'] = pd.to_datetime(feedback['feedback_date']).dt.to_period('M').astype(str)

# Initialize Dash app
app = dash.Dash(__name__)
server = app.server

# Create word cloud image
def create_wordcloud():
    text = " ".join(feedback['area_to_improve'].dropna())
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    img = BytesIO()
    wordcloud.to_image().save(img, format='PNG')
    return 'data:image/png;base64,{}'.format(base64.b64encode(img.getvalue()).decode())

# App layout
app.layout = html.Div([
    html.H1("Intern Performance Dashboard",style={'text-align': 'center'}),

    # Filters
    html.Div([
        dcc.Dropdown(
            id='dept-filter',
            options=[{'label': dept, 'value': dept} for dept in performance['department'].unique()],
            value=None,
            placeholder="Filter by Department",
            multi=True
        ),
        dcc.DatePickerRange(
            id='date-range',
            min_date_allowed=tasks['assigned_date'].min(),
            max_date_allowed=tasks['assigned_date'].max(),
            start_date=tasks['assigned_date'].min(),
            end_date=tasks['assigned_date'].max()
        )
    ], style={'width': '100%', 'display': 'flex', 'justify-content': 'space-between', 'margin': '20px 0'}),

    # Key Metrics Cards
    html.Div([
        html.Div([
            html.H3("Avg Completion Rate"),
            html.H2(id='avg-completion', style={'color': '#2ecc71'})
        ], className='metric-card'),

        html.Div([
            html.H3("Avg Quality Score"),
            html.H2(id='avg-quality', style={'color': '#3498db'})
        ], className='metric-card'),

        html.Div([
            html.H3("Avg Mentor Rating"),
            html.H2(id='avg-rating', style={'color': '#9b59b6'})
        ], className='metric-card'),

        html.Div([
            html.H3("Avg Completion Days"),
            html.H2(id='avg-days', style={'color': '#e74c3c'})
        ], className='metric-card')
    ], className='metrics-container'),

    # Main Visualizations
    html.Div([
        dcc.Graph(id='completion-distribution'),
        dcc.Graph(id='dept-performance')
    ], style={'display': 'flex'}),

    html.Div([
        dcc.Graph(id='time-trend'),
        dcc.Graph(id='priority-analysis')
    ], style={'display': 'flex'}),

    html.Div([
        dcc.Graph(id='quality-scores'),
        html.Div([
            html.H3("Common Improvement Areas"),
            html.Img(src=create_wordcloud(), style={'width': '100%'})
        ], style={'width': '50%', 'padding': '20px'})
    ], style={'display': 'flex'}),

    # Insights Section
    html.Div([
        html.H2("Key Insights", style={'margin-top': '40px'}),
        html.Ul(id='insights-list', style={'background': '#f8f9fa', 'padding': '20px', 'border-radius': '5px'})
    ])
], style={'font-family': 'Arial', 'max-width': '1400px', 'margin': '0 auto'})

# Callbacks for interactivity
@app.callback(
    [Output('avg-completion', 'children'),
     Output('avg-quality', 'children'),
     Output('avg-rating', 'children'),
     Output('avg-days', 'children'),
     Output('completion-distribution', 'figure'),
     Output('dept-performance', 'figure'),
     Output('time-trend', 'figure'),
     Output('priority-analysis', 'figure'),
     Output('quality-scores', 'figure'),
     Output('insights-list', 'children')],
    [Input('dept-filter', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date')]
)
def update_dashboard(selected_depts, start_date, end_date):
    # Filter data
    filtered_perf = performance.copy()
    filtered_tasks = tasks[(tasks['assigned_date'] >= start_date) &
                          (tasks['assigned_date'] <= end_date)]
    filtered_quality = quality.copy()
    filtered_feedback = feedback.copy()

    if selected_depts:
        filtered_perf = filtered_perf[filtered_perf['department'].isin(selected_depts)]
        intern_ids = filtered_perf['intern_id'].unique()
        filtered_tasks = filtered_tasks[filtered_tasks['intern_id'].isin(intern_ids)]
        filtered_quality = filtered_quality[filtered_quality['intern_id'].isin(intern_ids)]
        filtered_feedback = filtered_feedback[filtered_feedback['intern_id'].isin(intern_ids)]

    # Calculate metrics
    avg_completion = f"{filtered_perf['completion_rate'].mean():.1f}%"
    avg_quality = f"{filtered_quality['quality_score'].mean():.1f}/5"
    avg_rating = f"{filtered_feedback['rating'].mean():.1f}/5"
    avg_days = f"{filtered_tasks['days_to_complete'].mean():.1f} days"

    # Create visualizations
    completion_dist = px.histogram(
        filtered_perf, x='completion_rate', nbins=10,
        title='Task Completion Rate Distribution',
        labels={'completion_rate': 'Completion Rate (%)'}
    )

    dept_perf = px.box(
        filtered_perf, x='department', y='completion_rate',
        title='Department-wise Performance',
        color='department'
    )

    time_trend = px.line(
        filtered_tasks.groupby('month')['days_to_complete'].mean().reset_index(),
        x='month', y='days_to_complete',
        title='Average Completion Time Trend',
        markers=True
    )

    priority_analysis = px.box(
        filtered_tasks, x='priority', y='days_to_complete',
        title='Task Priority vs Completion Time',
        color='priority',
        category_orders={'priority': ['Low', 'Medium', 'High']}
    )

    quality_scores = px.histogram(
        filtered_quality, x='quality_score',
        title='Project Quality Score Distribution',
        nbins=5
    )

    # Generate insights
    insights = []

    # Insight 1: Top performing department
    best_dept = filtered_perf.groupby('department')['completion_rate'].mean().idxmax()
    insights.append(html.Li(f"🚀 {best_dept} department has the highest average completion rate"))

    # Insight 2: Quality vs Completion correlation
    merged_data = filtered_perf.merge(filtered_quality.groupby('intern_id')['quality_score'].mean(), on='intern_id')
    corr = merged_data[['completion_rate', 'quality_score']].corr().iloc[0,1]
    insights.append(html.Li(f"🔍 Moderate correlation between completion rate and quality scores (r = {corr:.2f})"))

    # Insight 3: Common improvement area
    common_issue = feedback['area_to_improve'].mode()[0]
    insights.append(html.Li(f"📌 Most frequent mentor feedback area: {common_issue}"))

    # Insight 4: Late tasks analysis
    late_tasks_pct = (filtered_tasks['status'] == 'Late Completion').mean() * 100
    insights.append(html.Li(f"⏱️ {late_tasks_pct:.1f}% of completed tasks were submitted late"))

    return (avg_completion, avg_quality, avg_rating, avg_days,
            completion_dist, dept_perf, time_trend, priority_analysis,
            quality_scores, insights)

if __name__ == '__main__':
    app.run(debug=True) # Changed from app.run_server to app.run

<IPython.core.display.Javascript object>